In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Tempo_run_2

/content/drive/MyDrive/Tempo_run_2


In [4]:
!zip -r tempo_run.zip /content/drive/MyDrive/Tempo_run_2

Scanning files .
  adding: content/drive/MyDrive/Tempo_run_2/ (stored 0%)
  adding: content/drive/MyDrive/Tempo_run_2/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/ (stored 0%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/train.txt (deflated 77%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/val_test.txt (deflated 61%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/spell_error.txt (deflated 37%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/train_pre.txt (deflated 75%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/new_data.txt (deflated 75%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/data_pre.txt (deflated 77%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/data_train_pre.txt (deflated 77%)
  adding: content/drive/MyDrive/Tempo_run_2/file_txt/data_phase_2.txt (deflated 66%)
  adding: content/drive/MyDrive/Tempo_ru

In [ ]:
!git init

Initialized empty Git repository in /content/drive/MyDrive/Tempo_run_2/.git/


In [ ]:
!git status

In [ ]:
!pip install transformers
!pip install underthesea
!pip install datasets

In [ ]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer, Trainer, DataCollatorWithPadding, TrainingArguments
import underthesea
import regex
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from datasets import load_dataset

###**Load pretrain**

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

###**Constant**

In [ ]:
label2id = {
    'negative': 0,
    'positive': 1,
    'neutral ': 2
}
id2label = {
    '0' : 'negative',
    '1' : 'positive',
    '2' : 'neutral'
}
labels_path = "file_txt/y_train.txt"
input_path = "file_txt/X_train.txt"
public_input_path = "file_txt/public_input.txt"
public_labels_path = "file_txt/public_labels.txt"
private_input_path = "file_txt/private_input.txt"
private_labels_path = "file_txt/private_labels.txt"
dataset_path = "file_csv/dataset.csv"
train_path = "file_csv/train.csv"
test_path = "file_csv/test.csv"
val_path = "file_csv/val.csv"
public_test_path = "file_csv/public_test.csv"
private_test_path = "file_csv/private_test.csv"

train_file_path = "./file_txt/train_pre.txt"
val_txt_path = 'file_txt/val_test.txt'
teencode_path = 'file_txt/teencode.txt'
spell_error_path = 'file_txt/spell_error.txt'
dataset_train_path = "file_csv/dataset_train.csv"
train_data_path = "file_csv/train_data.csv"
test_data_path = "file_csv/test_data.csv"
val_data_path = "file_csv/val_data.csv"
val_csv_path = 'file_txt/val_test.csv'

In [ ]:
f = open('/content/drive/MyDrive/Tempo_run_2/my_data.txt', 'r')
lines = f.readlines()
positive = []
negative = []
neutral = []
for line in lines:
  if(line[:8] == 'positive'):
    positive.append(line)
  elif(line[:8] == 'negative'):
    negative.append(line)
  elif(line[:8] == 'neutral '):
    neutral.append(line)
print(len(positive), len(negative), len(neutral))
with open('positive.txt', 'w+') as pos_file:
  for line in positive:
    pos_file.write(line)

with open('negative.txt', 'w+') as neg_file:
  for line in negative:
    neg_file.write(line)

with open('neutral.txt', 'w+') as neu_file:
  for line in neutral:
    neu_file.write(line)

###**Define method**

In [ ]:
def convert_train_data(path_file): # input: file_train, output: input and labels (type list)
  f = open(path_file, 'r')
  lines = f.readlines()
  labels = []
  input = []
  for line in lines:
    labels.append(label2id[line[:8]])
    line_split = regex.findall(r'(?i)\b\p{L}+\b', line[8:].replace('\t', '').replace('\n', '').lower())
    line_pre = " ".join(line_split)
    line_pre = underthesea.word_tokenize(line_pre, format="text")
    input.append(line_pre)
    # labels_pre, input_pre = line.replace('\n', '').split('\t')
    # input.append(input_pre)
    # labels.append(label2id[labels_pre])
  return input, labels

def convert_test_data(path_file, name_file):  # input: file_train, output: input (type list)
  f = open(path_file, 'r')
  lines = f.readlines()
  lines = change_teencode_spell_error(lines)
  input = []
  for line in lines:
    line_pre = underthesea.word_tokenize(line_pre, format="text")
    input.append(line_pre)
  print(len(input))
  df = pd.DataFrame(input, columns = ['content'])
  df.to_csv(name_file, index = False)

def convert_file_csv(input, labels, name_file): 
  df = pd.DataFrame([input, labels]).transpose()
  df.columns = ['content', 'labels']
  df.to_csv(name_file, index = False)
  print(len(input))
  print(len(labels))

def split_train_val_test(input, labels, ratio_train_val_test):
  x = sum(ratio_train_val_test[:2])/sum(ratio_train_val_test)
  y = ratio_train_val_test[0]/sum(ratio_train_val_test[:2])
  input_train_val, input_test, labels_train_val, labels_test = train_test_split(input, labels, train_size = x)
  input_train, input_val, labels_train, labels_val = train_test_split(input_train_val, labels_train_val, train_size = y)
  convert_file_csv(input_train, labels_train, train_data_path)
  convert_file_csv(input_val, labels_val, val_data_path)
  convert_file_csv(input_test, labels_test, test_data_path)

def compute_metrics(pred):
  labels = pred[1]
  preds = np.argmax(pred[0], axis = -1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average = "macro")
  acc = accuracy_score(labels, preds)
  return {
      "accuracy": acc,
      "f1": f1,
      "precision": precision,
      "recall": recall
  }

def tokenize_function(examples):
  return tokenizer(examples["content"], padding="max_length", truncation=True)

def get_tokenized_datasets(data_files, remove_columns):
  dataset = load_dataset("csv", data_files=data_files)
  tokenized_datasets = dataset.map(tokenize_function, batched=True)
  tokenized_datasets = tokenized_datasets.remove_columns(remove_columns)
  tokenized_datasets = tokenized_datasets.with_format('torch')
  print(tokenized_datasets)
  return tokenized_datasets

def change_teencode_spell_error(data):
  f = open(teencode_path, 'r')
  f1 = open(spell_error_path, 'r')
  list_teencode = f.readlines()
  list_se = f1.readlines()
  teencode = dict()
  se = dict()
  for line in list_teencode:
    a, b = line.replace('\n', '').split('\t')
    teencode[a] = b
  for line in list_se:
    a, b = a, b = line.replace('\n', '').split('\t')
    se[a] = b
  data_pre = []
  for ind, line in enumerate(data):
    line_split = line.split(' ')
    for i, token in enumerate(line_split):
      try:
        line_split[i] = teencode[token]
      except:
        if(token in se.keys()):
          line_split[i] = se[token]
    line_pre = ' '.join(line_split)
    data_pre.append(line_pre)
  return data_pre

In [ ]:
f = open('/content/drive/MyDrive/Tempo_run_2/file_txt/train.txt', 'r')
lines_input = f.readlines()
for ind, line in enumerate(lines_input):
  lines_input[ind] = line[8:].replace('\t', '').replace('\n', '')
print(lines_input[:10])

['tài liệu được bổ sung nhiều và đầy đủ trên moodle .', 'giảng viên phải nên cho ví dụ cụ thể những phần quan trọng nên nói kỹ có thể lấy ví dụ minh họa tránh việc lập đi lập lại nhiều lần sẽ tạo cảm giác chán cho sinh viên .', 'bài tập đa dạng , cụ thể .', 'slide up đầy đủ trước mỗi buổi học .', 'nội dung giảng dạy quá nhàm chán , khô khan .', 'thầy chép bảng nhiều .', 'thầy tận tình hướng dẫn cách làm , giải quyết những thắc mắc của sinh viên .', '  Rất ổn.', 'thầy dạy khá hay , cung cấp nhiều thông tin liên quan đến môn học , đưa ra nhiều hướng giải quyết khi sinh viên gặp vấn đề khó khăn với đồ án !', '  hy vọng học kỳ sau có thể học được với cô .']


In [ ]:
data_pre = change_teencode_spell_error(lines_input)

In [ ]:
with open('/content/drive/MyDrive/Tempo_run_2/file_txt/train_pre.txt', 'w+') as f:
  for line in data_pre:
    f.write(line + '\n')

###**Preprocess data**

In [ ]:
def get_data(path_file):
  df = pd.read_csv(path_file)
  input = df['content'].to_list()
  labels = df['labels'].to_list()
  return input, labels

In [ ]:
input_pos, labels_pos = get_data('positive_pre.csv')
input_neg, labels_neg = get_data('negative_pre.csv')
input_neu, labels_neu = get_data('neutral_pre.csv')

In [ ]:
print(len(input_pos), len(labels_pos))
print(len(input_neg), len(labels_neg))
print(len(input_neu), len(labels_neu)) 

8038 8038
7439 7439
698 698


In [ ]:
input = input_pos[1600:1701] + input_neg[1600:1701] + input_neu[600:]
labels = labels_pos[1600:1701] + labels_neg[1600:1701] + labels_neu[600:]
# input = input_pos + input_neg 
# labels = labels_pos + labels_neg 
ratio_train_val_test = [7, 1, 2]
split_train_val_test(input, labels, ratio_train_val_test)

210
210
30
30
60
60


In [ ]:
train_data_files = {"train": train_data_path, "test": test_data_path, "validation": val_data_path}
train_remove_columns = ["content"]
tokenized_datasets = get_tokenized_datasets(train_data_files, train_remove_columns)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-36093e80a01a82ed/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 210
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 60
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30
    })
})


###**Config trainer**

In [ ]:
path_checkpoint = '/content/drive/MyDrive/Tempo_run_2/train_test/train_1/checkpoint-81'
model = RobertaForSequenceClassification.from_pretrained(path_checkpoint)
# model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/Tempo_run_2/model/model_val_phase')

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)
training_args = TrainingArguments(output_dir="train_test/train_1",
                                  num_train_epochs = 3,
                                  learning_rate = 2e-5,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'steps',
                                  eval_steps = 40,
                                  save_steps = 81,
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

###**Training**

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
40,No log,0.058797,0.966667,0.952137,0.958333,0.952381
80,No log,0.206739,0.933333,0.916639,0.932773,0.904762


TrainOutput(global_step=81, training_loss=0.2033336074263961, metrics={'train_runtime': 38.5727, 'train_samples_per_second': 16.333, 'train_steps_per_second': 2.1, 'total_flos': 82880726584320.0, 'train_loss': 0.2033336074263961, 'epoch': 3.0})

###**Test**

In [ ]:
trainer.evaluate(tokenized_datasets['test'])

{'eval_loss': 0.3350745737552643,
 'eval_accuracy': 0.95,
 'eval_f1': 0.9478036175710595,
 'eval_precision': 0.9473684210526315,
 'eval_recall': 0.9545454545454546,
 'eval_runtime': 0.9726,
 'eval_samples_per_second': 61.688,
 'eval_steps_per_second': 8.225,
 'epoch': 3.0}

###**Predict**

In [ ]:
convert_test_data(val_txt_path, val_csv_path)

100


In [ ]:
data_files = {"val_test": val_csv_path}
remove_columns = ["content"]
tokenized_datasets_predict = get_tokenized_datasets(data_files, remove_columns)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating val_test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-de6e7aebb9230924/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    val_test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})


In [ ]:
predictions, _, _ = trainer.predict(tokenized_datasets_predict['val_test'])
labels_predict = np.argmax(predictions, axis=1)
print(len(labels_predict))

100


In [ ]:
with open("file_txt/train_pre.txt","w+") as file_submit:
  for ind, ip in enumerate(input):
    file_submit.write(id2label[str(labels_predict[ind])] + '\t' + ip + '\n')

In [ ]:
with open("submit/submit.txt","w+") as file_submit:
  for label in labels_predict:
    file_submit.write(id2label[str(label)] + '\n')

In [ ]:
trainer.save_model('model/model_val_phase')